In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev
import pandas as pd

In [ ]:
# Metrics helpers
import emeval.metrics.dist_calculations as emd
import emeval.metrics.reference_trajectory as emr

In [ ]:
# For computation
import numpy as np
import math
import scipy.stats as stats
import matplotlib.pyplot as plt

In [ ]:
import geopandas as gpd
import shapely as shp

In [ ]:
DATASTORE_URL = "http://cardshark.cs.berkeley.edu"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
sd_la = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "unimodal_trip_car_bike_mtv_la")
sd_sj = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "car_scooter_brex_san_jose")
sd_ucb = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "train_bus_ebike_mtv_ucb")

In [ ]:
import importlib
importlib.reload(ezpv)

In [ ]:
pv_la = eipv.PhoneView(sd_la)

In [ ]:
pv_sj = eipv.PhoneView(sd_sj)

In [ ]:
pv_ucb = eipv.PhoneView(sd_ucb)

### Simple wrapper functions

In [ ]:
def get_polygons(pvunp):
    """
    Return GeoSeries of polygons
    """
    polygons = []
    trips = pvunp.spec_details.curr_spec['evaluation_trips']

    for trip in trips:
        for leg in trip['legs']:
            if 'loc' in leg and leg['loc']['geometry']['type'] == 'Polygon':
                polygons.append(shp.geometry.Polygon(leg['loc']['geometry']['coordinates'][0]))
            if 'end_loc' in leg and leg['end_loc']['geometry']['type'] == 'Polygon':
                polygons.append(shp.geometry.Polygon(leg['end_loc']['geometry']['coordinates'][0]))
            if 'start_loc' in leg and leg['start_loc']['geometry']['type'] == 'Polygon':
                polygons.append(shp.geometry.Polygon(leg['start_loc']['geometry']['coordinates'][0]))
    return gpd.GeoSeries(polygons)

In [ ]:
def is_point_outside_polygons(loc_row, polygons):
    """
    Utility function to check if a point represented by a row in a location dataframe
    is contained within a series of Shapely polygons
    """
    # print(loc_row)
    point = loc_row.geometry
    inside_polygons = polygons.contains(point)
    return not inside_polygons.any()

def get_travel_trajectory(df, polygons):
    """ 
    Filters the dataframe of location points to only include values outside the defined polygons
    """
    geo_df = gpd.GeoDataFrame(
        df, geometry=df.apply(lambda lr: shp.geometry.Point(lr.longitude, lr.latitude), axis=1))
    geo_df["outside_polygons"] = geo_df.apply(lambda r: is_point_outside_polygons(r, polygons), axis=1)
    # return a slice instead of setting a column value
    return geo_df.query("outside_polygons==True")

def get_gt_linestring(gt_leg):
    """
    Get lat-long corrdinates in ground truth
    """
    if 'route_coords' in gt_leg:
        coords = gt_leg['route_coords']['geometry']['coordinates']
    else:
        coords = []
    return shp.geometry.LineString(coords)

## Spatial-temporal error calculation

In [ ]:
def get_reference_trajectory_input_tree(pv):
    ref_tree = {}
    # This is a GeoSeries
    # polygons = get_polygons(pv)
    
    for phone_os, phone_map in pv.map().items():
        for phone_label, phone_detail_map in phone_map.items():
            for (r_idx, r) in enumerate(phone_detail_map["evaluation_ranges"]):
                if r["eval_role_base"] != "accuracy_control":
                    continue
                for (tr_idx, tr) in enumerate(r["evaluation_trip_ranges"]):
                    for (sr_idx, sr) in enumerate(tr["evaluation_section_ranges"]):
                        # This is a Shapely LineString
                        section_gt_leg = pv.spec_details.get_ground_truth_for_leg(tr["trip_id_base"], sr["trip_id_base"])
                        section_gt_points = get_gt_linestring(section_gt_leg)
                        if section_gt_points.is_empty:
                            print("No ground truth route for %s %s, must be polygon, skipping..." % (tr["trip_id_base"], sr["trip_id_base"]))
                            assert section_gt_leg["type"] != "TRAVEL", "For %s, %s, %s, %s, %s found type %s" % (phone_os, phone_label, r_idx, tr_idx, sr_idx, section_gt_leg["type"])
                            continue
                        if len(sr['location_df']) == 0:
                            print("No sensed locations found, role = %s skipping..." % (r["eval_role_base"]))
                            # assert r["eval_role_base"] == "power_control", "Found no locations for %s, %s, %s, %s, %s" % (phone_os, phone_label, r_idx, tr_idx, sr_idx)
                            continue
                        
                        sec_name = tr["trip_id_base"] + "/" + sr["trip_id_base"] + "_" + str(r_idx)
                        if sec_name not in ref_tree:
                            ref_tree[sec_name] = {
                                "trip_id": tr["trip_id_base"],
                                "section_id": sr["trip_id_base"],
                                "run": r_idx,
                                "ground_truth": {
                                    "leg": section_gt_leg,
                                    "linestring": section_gt_points
                                }
                            }
                        
                        assert sec_name in ref_tree
                        e = ref_tree[sec_name]
                        # This is a GeoDataFrame
                        # section_measured_points = get_travel_trajectory(sr['location_df'], polygons)
                        section_measured_points = sr["location_df"]
                        if "temporal_control" not in e:
                            e["temporal_control"] = {}
                            e["start_ts"] = sr["start_ts"]
                            e["end_ts"] = sr["end_ts"]
                        e["temporal_control"][phone_os] = sr
    return ref_tree

def fill_ref_tree_entry(pv, e):
    print("Considering entry %s %s %s" % (e["trip_id"], e["section_id"], e["run"]))
    curr_tz = pv.spec_details.eval_tz
    assert "android" in e["temporal_control"] and "ios" in e["temporal_control"]
    e["reference_df"] = emr.final_ref_ensemble(e, 25, tz=curr_tz)

def get_reference_trajectory_tree(pv, ref_tree_root):
    curr_ref_tree = get_reference_trajectory_input_tree(pv)
    ref_tree_root[pv.spec_details.CURR_SPEC_ID] = curr_ref_tree
    [fill_ref_tree_entry(pv, e) for e in curr_ref_tree.values()]

In [ ]:
ref_tree = {}
get_reference_trajectory_tree(pv_la, ref_tree)
get_reference_trajectory_tree(pv_sj, ref_tree)
get_reference_trajectory_tree(pv_ucb, ref_tree)

In [ ]:
C = 40075017 # circumference of the earth in meters
def get_spatio_temporal_errors(pv):
    spatial_error_list = []
    # This is a GeoSeries
    polygons = get_polygons(pv)
    
    for phone_os, phone_map in pv.map().items():
        for phone_label, phone_detail_map in phone_map.items():
            for (r_idx, r) in enumerate(phone_detail_map["evaluation_ranges"]):
                run_errors = []
                for (tr_idx, tr) in enumerate(r["evaluation_trip_ranges"]):
                    trip_errors = []
                    for (sr_idx, sr) in enumerate(tr["evaluation_section_ranges"]):
                        # This is a Shapely LineString
                        section_gt_leg = pv.spec_details.get_ground_truth_for_leg(tr["trip_id_base"], sr["trip_id_base"])
                        section_gt_points = get_gt_linestring(section_gt_leg)
                        if section_gt_points.is_empty:
                            print("No ground truth route for %s %s, must be polygon, skipping..." % (tr["trip_id_base"], sr["trip_id_base"]))
                            assert section_gt_leg["type"] != "TRAVEL", "For %s, %s, %s, %s, %s found type %s" % (phone_os, phone_label, r_idx, tr_idx, sr_idx, section_gt_leg["type"])
                            continue
                        if len(sr['location_df']) < 2:
                            print("No sensed locations found, role = %s skipping..." % (r["eval_role_base"]))
                            # assert r["eval_role_base"] == "power_control", "Found no locations for %s, %s, %s, %s, %s" % (phone_os, phone_label, r_idx, tr_idx, sr_idx)
                            continue

                        # This is a GeoDataFrame
                        eval_df = emr.get_int_aligned_trajectory(sr['location_df'], tz=pv.spec_details.eval_tz)
                        sr["resampled_df"] = eval_df
                        spec_name = tr["trip_id_base"]+"/"+sr["trip_id_base"]+"_"+str(r_idx)
                        reference_df = ref_tree[pv.spec_details.CURR_SPEC_ID][spec_name]["reference_df"]
                        print("len(eval_df) = %d, len(reference_df) = %d" % (len(eval_df), len(reference_df)))
                        # Match these up by timestamp
                        merged_df = pd.merge(eval_df, reference_df, on="ts", how="inner", suffixes=("_e", "_r")).sort_values(by="ts", axis="index")
                        merged_df["t_distance"] = gpd.GeoSeries(merged_df.geometry_e).distance(gpd.GeoSeries(merged_df.geometry_r)) * (C/360)
                        spatial_error_entries = [{"phone_os": phone_os, "phone_label": phone_label,
                                               "timeline": pv.spec_details.CURR_SPEC_ID,
                                               "run": r_idx, "role": r["eval_role_base"],
                                               "trip_id": tr["trip_id_base"], "section_id": sr["trip_id_base"],
                                               "error": e} for e in merged_df.t_distance]
                        spatial_error_list.extend(spatial_error_entries)
    return spatial_error_list

In [ ]:
st_errors_list = []
st_errors_list.extend(get_spatio_temporal_errors(pv_la))
st_errors_list.extend(get_spatio_temporal_errors(pv_sj))
st_errors_list.extend(get_spatio_temporal_errors(pv_ucb))

st_errors_df = pd.DataFrame(st_errors_list)

In [ ]:
r2q_map = {"power_control": 0, "HAMFDC": 1, "MAHFDC": 2, "HAHFDC": 3, "accuracy_control": 4}
q2r_map = {0: "power", 1: "HAMFDC", 2: "MAHFDC", 3: "HAHFDC", 4: "accuracy"}

In [ ]:
st_errors_df["quality"] = st_errors_df.role.apply(lambda r: r2q_map[r])
st_errors_df["label"] = st_errors_df.role.apply(lambda r: r.replace('_control', ''))
timeline_list = ["train_bus_ebike_mtv_ucb", "car_scooter_brex_san_jose", "unimodal_trip_car_bike_mtv_la"]

In [ ]:
st_errors_df.head()

In [ ]:
st_errors_df.query('timeline == "train_bus_ebike_mtv_ucb" & run == 0 & phone_os == "android" & quality == 0').error.head()

In [ ]:
ifig, ax_array = plt.subplots(nrows=2,ncols=3,figsize=(12,6), sharex=False, sharey=False)
timeline_list = ["train_bus_ebike_mtv_ucb", "car_scooter_brex_san_jose", "unimodal_trip_car_bike_mtv_la"]
for i, tl in enumerate(timeline_list):
    st_errors_df.query("timeline == @tl & phone_os == 'android' & quality > 0").boxplot(ax = ax_array[0][i], column=["error"], by=["quality"])
    ax_array[0][i].set_title(tl)
    st_errors_df.query("timeline == @tl & phone_os == 'ios' & quality > 0").boxplot(ax = ax_array[1][i], column=["error"], by=["quality"])
    ax_array[1][i].set_title("")

for i, ax in enumerate(ax_array[0]):
    ax.set_xticklabels([q2r_map[int(t.get_text())] for t in ax.get_xticklabels()])
    ax.set_xlabel("")

for i, ax in enumerate(ax_array[1]):
    ax.set_xticklabels([q2r_map[int(t.get_text())] for t in ax.get_xticklabels()])
    ax.set_xlabel("")

ax_array[0][0].set_ylabel("Battery drain (android)")
ax_array[1][0].set_ylabel("Battery drain (iOS)")
ifig.suptitle("Power v/s quality over multiple timelines")
# ifig.tight_layout()

In [ ]:
ifig, ax_array = plt.subplots(nrows=2,ncols=4,figsize=(25,10), sharex=True, sharey=True)
timeline_list = ["train_bus_ebike_mtv_ucb"]
for i, tl in enumerate(timeline_list):
    for q in range(1,5):
        sel_df = st_errors_df.query("timeline == @tl & phone_os == 'android' & quality == @q")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i][q-1], column=["error"], by=["section_id"])
        ax_array[2*i][q-1].tick_params(axis="x", labelrotation=45)
        sel_df = st_errors_df.query("timeline == @tl & phone_os == 'ios' & quality == @q")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i+1][q-1], column=["error"], by=["section_id"])
#        ax_array[i][].set_title("")

def make_acronym(s):
    ssl = s.split("_")
    # print("After splitting %s, we get %s" % (s, ssl))
    if len(ssl) == 0 or len(ssl[0]) == 0:
        return ""
    else:
        return "".join([ss[0] for ss in ssl])

for q in range(1,5):
    ax_array[0][q-1].set_title(q2r_map[q])
    curr_ticks = [t.get_text() for t in ax_array[1][q-1].get_xticklabels()]
    new_ticks = [make_acronym(t) for t in curr_ticks]
    ax_array[1][q-1].set_xticklabels(new_ticks)
    
print(list(zip(curr_ticks, new_ticks)))
# fig.text(0,0,"%s"% list(zip(curr_ticks, new_ticks)))

In [ ]:
timeline_list = ["train_bus_ebike_mtv_ucb"]
for i, tl in enumerate(timeline_list):
    unique_sections = st_errors_df.query("timeline == @tl").section_id.unique()
    ifig, ax_array = plt.subplots(nrows=2,ncols=len(unique_sections),figsize=(40,10), sharex=True, sharey=False)
    for sid, s_name in enumerate(unique_sections):
        sel_df = st_errors_df.query("timeline == @tl & phone_os == 'android' & section_id == @s_name & quality > 0")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i][sid], column=["error"], by=["quality"])
        ax_array[2*i][sid].set_title(s_name)
        sel_df = st_errors_df.query("timeline == @tl & phone_os == 'ios' & section_id == @s_name & quality > 0")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*i+1][sid], column=["error"], by=["quality"])
        ax_array[2*i+1][sid].set_title("")
#        ax_array[i][].set_title("")


### Focus only on sections where the max error is > 100 meters

In [ ]:
timeline_list = ["train_bus_ebike_mtv_ucb"]
for i, tl in enumerate(timeline_list):
    unique_sections = pd.Series(st_errors_df.query("timeline == @tl").section_id.unique())
    sections_with_outliers_mask = unique_sections.apply(lambda s_name: st_errors_df.query("timeline == 'train_bus_ebike_mtv_ucb' & section_id == @s_name").error.max() > 100)
    sections_with_outliers = unique_sections[sections_with_outliers_mask]   
    ifig, ax_array = plt.subplots(nrows=4,ncols=len(sections_with_outliers)//2,figsize=(16,12), sharex=True, sharey=False)
    for sid, s_name in enumerate(sections_with_outliers):
        sel_df = st_errors_df.query("timeline == @tl & phone_os == 'android' & section_id == @s_name & quality > 0")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*(sid//6)][sid%6], column=["error"], by=["quality"])
        ax_array[2*(sid//6)][sid%6].set_title(s_name)
        sel_df = st_errors_df.query("timeline == @tl & phone_os == 'ios' & section_id == @s_name & quality > 0")
        if len(sel_df) > 0:
            sel_df.boxplot(ax = ax_array[2*(sid//6)+1][sid%6], column=["error"], by=["quality"])
        ax_array[2*(sid//6)+1][sid%6].set_title("")
#        ax_array[i][].set_title("")
    ifig.suptitle("")

### Validation of outliers

#### (express bus iOS, MAHFDC)

ok, so it looks like the error is non-trivial across all runs, but run #1 is the worst and is responsible for the majority of the outliers. And this is borne out by the map, where on run #1, we end up with points in San Leandro!!

In [ ]:
st_errors_df.query("phone_os == 'ios' & quality == 2 & section_id == 'express_bus' & error > 500").run.unique()

In [ ]:
st_errors_df.query("phone_os == 'ios' & quality == 2 & section_id == 'express_bus'").boxplot(column="error", by="run")

In [ ]:
ucb_and_back = pv_ucb.map()["ios"]["ucb-sdb-ios-3"]["evaluation_ranges"][1]; ucb_and_back["trip_id"]

In [ ]:
back_trip = ucb_and_back["evaluation_trip_ranges"][2]; print(back_trip["trip_id"])

In [ ]:
eb_leg = back_trip["evaluation_section_ranges"][4]; print(eb_leg["trip_id"])

In [ ]:
gt_leg = sd_ucb.get_ground_truth_for_leg(back_trip["trip_id_base"], eb_leg["trip_id_base"]); gt_leg["id"]

In [ ]:
import folium

In [ ]:
importlib.reload(emr)

In [ ]:
curr_map = folium.Map()
gt_leg_gj = ezpv.get_geojson_for_loc_df(ref_tree["train_bus_ebike_mtv_ucb"]["berkeley_to_mtv_SF_express_bus/express_bus_1"]["reference_df"], color="green")
sensed_section_gj = ezpv.get_geojson_for_loc_df(eb_leg["resampled_df"])
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="reference")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj, name="reference_points", color="green")
sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="sensed_points", color="red", tz="America/Los_Angeles")
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
curr_map.add_child(sensed_leg_gj_feature)
curr_map.add_child(sensed_leg_gj_points)
curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

In [ ]:
curr_map = folium.Map()
gt_leg_gj = ezpv.get_geojson_for_loc_df(ref_tree["train_bus_ebike_mtv_ucb"]["berkeley_to_mtv_SF_express_bus/express_bus_1"]["reference_df"], color="green")
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="reference")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj, name="reference_points", color="green")

colors = ["red", "yellow", "blue"]
for run in range(3):
    ucb_and_back = pv_ucb.map()["ios"]["ucb-sdb-ios-3"]["evaluation_ranges"][run]; ucb_and_back["trip_id"]
    back_trip = ucb_and_back["evaluation_trip_ranges"][2]; print(back_trip["trip_id"])
    eb_leg = back_trip["evaluation_section_ranges"][4]; print(eb_leg["trip_id"])
    sensed_section_gj = ezpv.get_geojson_for_leg(eb_leg)
    sensed_section_gj["properties"]["style"]["color"] = colors[run]
    sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="run %s" % run)
    sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="points for %d" % run, color=colors[run], tz="America/Los_Angeles")
    curr_map.add_child(sensed_leg_gj_feature)
    curr_map.add_child(sensed_leg_gj_points)
    
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
curr_map.fit_bounds(gt_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

#### (commuter rail aboveground android, HAMFDC)

Runs along El Camino instead of the Caltrain tracks for a while. Not sure if this is even an outlier.

In [ ]:
st_errors_df.query("phone_os == 'android' & quality == 1 & section_id == 'commuter_rail_aboveground' & error > 500").run.unique()

In [ ]:
ucb_and_back = pv_ucb.map()["android"]["ucb-sdb-android-3"]["evaluation_ranges"][0]; ucb_and_back["trip_id"]
to_trip = ucb_and_back["evaluation_trip_ranges"][0]; print(to_trip["trip_id"])
train_leg = to_trip["evaluation_section_ranges"][2]; print(train_leg["trip_id"])
gt_leg = sd_ucb.get_ground_truth_for_leg(to_trip["trip_id_base"], train_leg["trip_id_base"]); gt_leg["id"]

In [ ]:
curr_map = folium.Map()
gt_leg_gj = ezpv.get_geojson_for_loc_df(ref_tree["train_bus_ebike_mtv_ucb"]["mtv_to_berkeley_sf_bart/commuter_rail_aboveground_0"]["reference_df"], color="green")
sensed_section_gj = ezpv.get_geojson_for_leg(train_leg)
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="reference")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj, name="reference_points", color="green")
sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="sensed_points", color="red", tz="America/Los_Angeles")
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
curr_map.add_child(sensed_leg_gj_feature)
curr_map.add_child(sensed_leg_gj_points)
curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

In [ ]:
unique_sections = pd.Series(st_errors_df.query("timeline == 'train_bus_ebike_mtv_ucb'").section_id.unique())
sections_with_outliers_mask = unique_sections.apply(lambda s_name: st_errors_df.query("timeline == 'train_bus_ebike_mtv_ucb' & section_id == @s_name").error.max() > 100)
unique_sections[sections_with_outliers_mask]

#### (walk_to_bus android, HAMFDC, HAHFDC)

In the spatial-only accuracy, run 0 was the worst, with a zig-zag out to San Francisco. But the error magnitude was only ~ 3k since it wasn't that far from BART. Now that we account for temporal differences, the error is much larger (9k) but it is only a zig zag out to Ashby. I am guessing that the other error got stripped out because there was no matching timestamp. Not going to worry about that for now...

In [ ]:
st_errors_df.query("phone_os == 'android' & (quality == 1 | quality == 3) & section_id == 'walk_to_bus' & error > 500").run.unique()

In [ ]:
st_errors_df.query("phone_os == 'android' & (quality == 1 | quality == 3) & section_id == 'walk_to_bus' & error > 500")

In [ ]:
st_errors_df.query("phone_os == 'android' & (quality == 1 | quality == 3) & section_id == 'walk_to_bus'").boxplot(column="error", by="run")

In [ ]:
ucb_and_back = pv_ucb.map()["android"]["ucb-sdb-android-2"]["evaluation_ranges"][1]; ucb_and_back["trip_id"]
to_trip = ucb_and_back["evaluation_trip_ranges"][0]; print(to_trip["trip_id"])
wb_leg = to_trip["evaluation_section_ranges"][6]; print(wb_leg["trip_id"])
gt_leg = sd_ucb.get_ground_truth_for_leg(to_trip["trip_id_base"], wb_leg["trip_id_base"]); gt_leg["id"]

In [ ]:
import folium

In [ ]:
curr_map = folium.Map()
gt_leg_gj = ezpv.get_geojson_for_loc_df(ref_tree["train_bus_ebike_mtv_ucb"]["mtv_to_berkeley_sf_bart/walk_to_bus_0"]["reference_df"], color="green")
sensed_section_gj = ezpv.get_geojson_for_leg(wb_leg)
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj, name="ground_truth_points", color="green")
sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="sensed_points", color="red", tz="America/Los_Angeles")
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
curr_map.add_child(sensed_leg_gj_feature)
curr_map.add_child(sensed_leg_gj_points)
curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

#### (light_rail_below_above_ground, android, accuracy_control)

In this case, the spatial error was bad, but the temporal error is not that terrible, mainly because all the 

In [ ]:
st_errors_df.query("phone_os == 'android' & quality == 4 & section_id == 'light_rail_below_above_ground' & error > 100").run.unique()

In [ ]:
st_errors_df.query("phone_os == 'android' & (quality == 4) & section_id == 'light_rail_below_above_ground'").boxplot(column="error", by="run")

In [ ]:
ucb_and_back = pv_ucb.map()["android"]["ucb-sdb-android-2"]["evaluation_ranges"][2]; ucb_and_back["trip_id"]
back_trip = ucb_and_back["evaluation_trip_ranges"][2]; print(back_trip["trip_id"])
lt_leg = back_trip["evaluation_section_ranges"][7]; print(lt_leg["trip_id"])
gt_leg = sd_ucb.get_ground_truth_for_leg(back_trip["trip_id_base"], lt_leg["trip_id_base"]); gt_leg["id"]

In [ ]:
import folium

In [ ]:
curr_map = folium.Map()
gt_leg_gj = ezpv.get_geojson_for_loc_df(ref_tree["train_bus_ebike_mtv_ucb"]["berkeley_to_mtv_SF_express_bus/light_rail_below_above_ground_0"]["reference_df"], color="green")
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj, name="ground_truth_points", color="green")

colors = ["red", "yellow", "blue"]
for run in range(3):
    ucb_and_back = pv_ucb.map()["android"]["ucb-sdb-android-2"]["evaluation_ranges"][run]; ucb_and_back["trip_id"]
    back_trip = ucb_and_back["evaluation_trip_ranges"][2]; print(back_trip["trip_id"])
    lt_leg = back_trip["evaluation_section_ranges"][7]; print(lt_leg["trip_id"])
    sensed_section_gj = ezpv.get_geojson_for_leg(lt_leg)
    sensed_section_gj["properties"]["style"]["color"] = colors[run]
    sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="run %s" % run)
    sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="points for %d" % run, color=colors[run], tz="America/Los_Angeles")
    curr_map.add_child(sensed_leg_gj_feature)
    curr_map.add_child(sensed_leg_gj_points)
    
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
curr_map.fit_bounds(gt_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

#### (subway, android, HAMFDC)

This is the poster child for temporal accuracy tracking

In [ ]:
bart_leg = pv_ucb.map()["android"]["ucb-sdb-android-3"]["evaluation_ranges"][0]["evaluation_trip_ranges"][0]["evaluation_section_ranges"][5]
gt_leg = sd_ucb.get_ground_truth_for_leg("mtv_to_berkeley_sf_bart", "subway_underground"); gt_leg["id"]

In [ ]:
curr_map = folium.Map()
gt_leg_gj = ezpv.get_geojson_for_loc_df(ref_tree["train_bus_ebike_mtv_ucb"]["mtv_to_berkeley_sf_bart/subway_underground_0"]["reference_df"], color="green")
sensed_section_gj = ezpv.get_geojson_for_leg(bart_leg)
gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj, name="ground_truth_points", color="green")
sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values")
sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="sensed_points", color="red", tz="America/Los_Angeles")
curr_map.add_child(gt_leg_gj_feature)
curr_map.add_child(gt_leg_gj_points)
curr_map.add_child(sensed_leg_gj_feature)
curr_map.add_child(sensed_leg_gj_points)
curr_map.fit_bounds(sensed_leg_gj_feature.get_bounds())
folium.LayerControl().add_to(curr_map)
curr_map

In [ ]:
bart_leg["location_df"].iloc[70:80]

In [ ]:
st_errors_df.query("phone_os == 'android' & (quality == 4) & section_id == 'subway_underground'").boxplot(column="error", by="run")